
# COVID-Team-1
### Dr. Mohanty
### Christopher Brantley
### CSC 405 01
### Last Updated: 10/17/2020
### I, Christopher Brantley, have abided by the UNCG academic policy.

In [1]:
import os
os.chdir("../")

In [5]:
import ipynb
from ipynb.fs.full import covid_data as cd
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [6]:
# Telling pandas not convert to html tags.
pd.set_option('display.html.table_schema', True)
# Max columns and rows to display.
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 5)

### Generate weekly statistics (mean, median, mode) for number of cases and deaths across a specific state.


In [75]:
df_primary_data = cd.get_covid_primary_data()
df_primary_cases = df_primary_data[df_primary_data.State == "FL"].filter(regex = "County|State|case")
number_days = df_primary_cases.filter(regex = "cases").shape[1]
df_florida_cases = pd.DataFrame(index = ["week " + str(day) for day in range(1, (number_days // 7) + 1, 1)],
                                columns = ["sum"])
week_count = 0

for lower_range in range(2, df_primary_cases.filter(regex = "cases").shape[1], 7):
    df_florida_cases.iloc[week_count] = df_primary_cases.iloc[:, lower_range:lower_range + 7].sum().sum()
    week_count += 1
display(df_florida_cases)

,sum
week 1,0
week 2,0
...,...
week 33,4490829
week 34,4617459


In [177]:
df_primary_data = cd.get_covid_primary_data()
number_days = df_primary_data.filter(regex = "cases").shape[1]
number_weeks = (number_days // 7) + 1
index_state = []
index_week = []
for state in df_primary_data.State.unique().tolist():
    for week in range(1, number_weeks, 1):
        index_state.append(state)
        index_week.append("week " + str(week))
df_weekly_cases = pd.DataFrame(index = [index_state, index_week],
                               columns = ["mean", "median", "mode"])
for state in df_weekly_cases.index.get_level_values(0).unique():
    df_state = df_primary_data[df_primary_data.State == state]
    for week in range(0, number_weeks - 1, 1):
        df_day = df_state.filter(regex = "cases")\
                .iloc[: , (week * 7) + 7]
        df_weekly_cases.loc[(state, ("week " + str(week + 1))), "mean"] = df_day.mean()
        df_weekly_cases.loc[(state, ("week " + str(week + 1))), "median"] = df_day.median()
        df_weekly_cases.loc[(state, ("week " + str(week + 1))), "mode"] = df_day.mode().mode()[0]
display(df_weekly_cases)

mean median mode
AL week 1         0      0    0
   week 2         0      0    0
...             ...    ...  ...
WY week 33  180.478    120   51
   week 34  198.522    127    3

[1734 rows x 3 columns]

### Compare the data against other states.

### Identify counties with high case and death rates.

### Plot daily trends (cases and deaths, new cases) of state and top 5 infected counties. Utilize aggregrate, normalized by population, and log normalized values.